In [8]:
# Import necessary libraries
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [9]:
# Initialize the OpenAI client, pointing to the local Ollama instance
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [10]:
 # --- First interaction with the Llama model ---

In [11]:
# Define a simple message for the Llama model
message = "Hello, Llama! This is my first ever message to you! Hi!"
# Send the message to the "llama3.2" model and get the response
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])


In [12]:
 # Print the content of the model's response
print(response.choices[0].message.content)

Hello! Welcome! I'm thrilled to be your first conversational partner! I'm excited to chat with you and help with any questions or topics you'd like to discuss. How's your day going so far?


In [13]:
# Define a common User-Agent header to avoid being blocked by websites
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
# --- Class definition for representing a Website ---
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# Create a Website object for the specified URL
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [15]:
# --- Defining the system prompt for the summarization task ---
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# --- Function to generate the user prompt based on the Website object ---
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [17]:
 # Print the generated user prompt for the example website
 print("Generated user prompt:\n")
 print(user_prompt_for(ed))

Generated user prompt:

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of 

In [18]:
# --- Function to structure the messages for the OpenAI API ---
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [19]:
# Text to separate the prompt definition from the summarization function

 

 # --- Function to summarize the content of a given URL ---

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
 # Summarize the example website and print the summary
summarize("https://edwarddonner.com")

'## Summary of Website "Home - Edward Donner"\n\n### Overview\n\nThe website is hosted by Edward Donner, a co-founder and CTO of Nebula.io, an AI company aiming to apply AI for talent discovery. The site introduces him personally, mentioning his background in coding, DJing, and participating in Hacker News.\n\n### LLM Workshop and Resources\n\nEdward Donner announces upcoming workshops focused on Large Language Models (LLMs), including:\n* January 23, 2025: LLM Workshop – Hands-on with Agents\n\t+ With provided resources available for attendees.\n* December 21, 2024: Welcome, SuperDataScientists!\n\t+ An announcement with a message directed to data scientists.\n\n### Recent News and Announcements\n\nThe website displays announcements from previous dates (November 13, 2024 to October 16, 2024). These include resources and information on mastering AI and LLM engineering.\n\n### Contact Information\n\nEdward Donner provides contact details using "ed" at edwarddonner.com for emails.'

In [21]:
 # --- Function to display the summary in Markdown format ---
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
# Display the summary of the example website using Markdown
display_summary("https://edwarddonner.com")

# Website Summary: Edward Donner's Home Page
=============================

The website is a personal homepage of Ed, the founder and CTO of Nebula.io. It appears that Ed is passionate about writing code and experimenting with Large Language Models (LLMs).

## Featured Section
-----------------

*   Description: Explains Ed's background in coding, his interests in DJing and amateur electronic music production.
*   Links to:
    *   Nebula.io (Ed's current position as CTO)
    *   untapt (acquired AI startup founded by Ed)

## Announcements & News
----------------------

### Recent Posts:
1. **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources
2. **December 21, 2024**: Welcome, SuperDataScientists!
3. **November 13, 2024**: Mastering AI and LLM Engineering – Resources
4. **October 16, 2024**: From Software Engineer to AI Data Scientist - resources

## Blog Categories:
------------------

*   Agent-based Systems
*   AI & LLMs
*   Code Snippets